In [1]:
using Distributed
# @everywhere used for the implementation of the modules across multiple workers
# Without it, the code does not run

# Add the modules to the ´read´ path of the Julia
@everywhere push!(LOAD_PATH, "/scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/")


# Custom modules
@everywhere using Model: HapticNet, VisualNet, train_epoch!, save_as_jld2
@everywhere using TUM69: load_accel_data, load_image_data
@everywhere using Preprocess: process_accel_signal, process_image, augment_image
@everywhere using Utils: kfold, notify!, a_type
# Ready-to-use modules
using JLD2, Random
using CUDA
using Augmentor: FlipX, FlipY

┌ Info: Precompiling Preprocess [top-level]
└ @ Base loading.jl:1278


In [2]:
# Set path
path = CUDA.functional() ? "/userfiles/vaydingul20/data/new" : "data/new" # path of the main data
DATA_PATH = isdir(path) && path
# Set pretrained network path
alexnet_PATH = "/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

"/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

In [3]:
# Load image data
X_train, y_train, _, _, material_dict = @time load_image_data(DATA_PATH; mode="normal");

Train image data is being loaded!
Test image data is being loaded!
479.786710 seconds (5.03 M allocations: 92.677 GiB, 2.03% gc time)


In [5]:
# Augmentation pipeline
p1 = FlipX()
p2 = FlipY()
p3 = FlipX() |> FlipY()
X_train, y_train = @time augment_image(X_train, y_train, p1, p2, p3);

LoadError: MethodError: no method matching toaffinemap(::Augmentor.NoOp, ::SubArray{Float32,3,Array{Float32,4},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Int64},true})
Closest candidates are:
  toaffinemap(::Augmentor.Operation, ::Any, !Matched::Any) at /kuacc/users/vaydingul20/.julia/packages/Augmentor/FMjHq/src/operation.jl:48
  toaffinemap(::Augmentor.NoOp, !Matched::AbstractArray{T,2} where T) at /kuacc/users/vaydingul20/.julia/packages/Augmentor/FMjHq/src/operations/noop.jl:17
  toaffinemap(!Matched::Augmentor.Either, ::Any, !Matched::Any) at /kuacc/users/vaydingul20/.julia/packages/Augmentor/FMjHq/src/operations/either.jl:170

In [4]:
# Apply preprocessing on the images
# Here is where the code explodes
X_train, y_train = @time process_image(X_train, y_train);

204.316427 seconds (12.28 M allocations: 11.306 GiB, 7.91% gc time)


In [ ]:
# Seperate into 3 folds for training
kf = kfold(X_train, y_train; fold=3, minibatch_size = 10, atype=a_type(Float32))
results = []

In [ ]:
for (ix, (dtrn, dtst)) in enumerate(kf.folds[1:1])
    

    #Reset the model
    vn = VisualNet(alexnet_PATH; atype=a_type(Float32))

    for k in 1:1
        # Training routine
        res = train_epoch!(vn, dtrn, dtst; progress_bar=true, fig=false, info=true, epoch=100)
        # Save model 
        # Add results to the ´results´vector
        push!(results, res)

    end
    save_as_jld2(vn, "vn-$ix.jld2")

end

In [ ]:
# Save accumulated results
JLD2.@save "results_vn.jld2" results = results